<a href="https://colab.research.google.com/github/vachkim/LECTURE/blob/master/MINING/Lec_MOVIE_REVIEW_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.데이터 불러오기
- [링크](https://github.com/e9t/nsmc)에서 리뷰데이터를 가져와 구글에 업로드
- Mount를 통해 불러온다

In [ ]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd drive/'My Drive'/'Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [ ]:
%ls

'모두의 데이터 분석'/          machinelearning_data/
 0826_Class.ipynb              Matplotlib_시각화.ipynb
 강의_BeautifulSoup.ipynb      naivebayes_example.csv
 deeplearning_data/            ratings_test.txt
 강의_판다스.ipynb             ratings_train.txt
 강의_파이썬.ipynb             SamsungCard_분석.ipynb
 강의_넘파이.ipynb             SamsungCard.ipynb
 강의_코딩실습.ipynb           samsung_data/
 강의_영어마이닝.ipynb         Seaborn_시각화.ipynb
 강의_감정분석.ipynb           강의_Selenium.ipynb
 강의_한글마이닝.ipynb         titanic_data/
 강의_영화리뷰감정분석.ipynb   Titanic_기초분석.ipynb
 데이터시각화기초.ipynb        Untitled
 인구공공데이터.ipynb          weather_data/
 기온공공데이터.ipynb


In [ ]:
import pandas as pd
test = pd.read_csv('ratings_test.txt',sep='\t',quoting=3)
train = pd.read_csv('ratings_train.txt',sep='\t',quoting=3)

- 데이터를 보면 아이디와 후기, 후기에 대한 긍정/부정 평가로 세트가 구성되어 있음을 알 수 있다

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
train.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


- 원활한 분석을 위해 null값을 지워준다

In [ ]:
train.isnull().sum()

id          0
document    5
label       0
dtype: int64

- 결측값 5개를 지우고 난 뒤 전체 데이터셋의 갯수가 14만 9995개가 된 것을 확인할 수 있다


In [ ]:
train = train.dropna()

## 2.텍스트전처리
- 위에 리뷰를 보면 온전하지 않은 단어들이 등장함을 알 수 있다
- 감정분석 패키지에 넣기 적절한 형태로 데이터의 전처리를 수행한다

### 1) 토큰화
- Okt는 품사 단위로 토큰화 할 뿐만 아니라 SNS등에서 사용되는 이모티콘을 구분할 수 있다
- 이에 여기에서는 Okt를 가지고 토큰화를 진행하도록 한다

In [ ]:
!pip install konlpy
from konlpy.tag import Okt

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 6.7MB/s 
     |████████████████████████████████| 3.8MB 47.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


- Okt는 입력값으로 하나의 STR 값을 받는다
- 따라서 Document칼럼의 값을 문자열로 변환하는 과정이 필요하다

In [ ]:
all = list(train['document'])

In [ ]:
review = ''
for element in all:
  review += element

In [ ]:
review[:500]

'아 더빙.. 진짜 짜증나네요 목소리흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나너무재밓었다그래서보는것을추천한다교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.원작의 긴장감을 제대로 살려내지못했다.별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네액션이 없는데도 재미 있는 몇안되는 영화왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?걍인피니트가짱이다.진짜짱이다♥볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해담백하고 깔끔해서 좋다. 신문기사로만 보다 보면 자'

In [ ]:
okt = Okt()
okt_tokens = okt.morphs(review)

### 2) 품사부착
- 토큰화된 단어에 대해 품사를 부착한다

In [ ]:
oktTag = []
for token in okt_tokens:
  oktTag += okt.pos(token)
print(oktTag)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### 3) 불용어처리
- 불필요한 품사/단어를 제거하여 감정분석의 정확도를 높인다
- 품사 정보는 [다음](https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0)을 참고하자
- Counter를 사용해 토큰화된 품사/단어들을 확인, 불용어 처리를 진행한다

In [ ]:
from collections import Counter
Counter(oktTag).most_common()

In [ ]:
stopPos = ['Conjunction','Josa','PreEomi','Eomi','Suffix','Punctuation','Foreign','Alpha','Number','Unknown',
           'KoreanParticle','Hashtag','ScreeName','Email','URL']
stopWord = ['네','이','의','가','더','때','고','도','들','는','를','은','임','이런','이나','부터','저','게','수',
            '한','다','만','기','정','도','과','것','요','서','면','지','입니다','대','거','듯','내','그','나',
            '인','안','와','결','비','하는데','머','까','조','건가','하면서','대해','했던','될','뒤']

In [ ]:
X_train = []
for sentence in train['document']:
    temp_X = []
    temp_X = okt.pos(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopWord]

In [ ]:
stopPos = ['Conjunction','Josa','PreEomi','Eomi','Suffix','Punctuation','Foreign','Alpha','Number','Unknown',
           'KoreanParticle','Hashtag','ScreeName','Email','URL']
stopWord = ['네','이','의','가','더','때','고','도','들','는','를','은','임','이런','이나','부터','저','게','수',
            '한','다','만','기','정','도','과','것','요','서','면','지','입니다','대','거','듯','내','그','나',
            '인','안','와','결','비','하는데','머','까','조','건가','하면서','대해','했던','될','뒤']
for tag in oktTag:
  if tag[1] not in stopPos:
    if tag[0] not in stopWord:
      word.append(tag[0])

In [ ]:
X_train = word
Y_train = train['label']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
clf = MultinomialNB().fit(X_train, Y_train)